In [1]:
from Bio import SeqIO
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import time
import seaborn as sns

%matplotlib inline

### Evaluate the time difference req'd to run minimap2 against the initial v. new .fasta file

In [2]:
# take the outputs of experiment-4 -- {this_dump}.fasta and {this_dump}.new.fasta -- and use these
# this cell will maintain parity with the dumps generated for experiment-4

this_dump = "../experiment-4/moraxellacatarhalis_29497_dump"

#this_dump = "../experiment-4/candidaauris_498019_dump"
#this_dump = "../experiment-4/rhinovirusc_463676_dump"
#this_dump = "../experiment-4/chkv_txid37124_dump"

# additionally, set a reference genome from which to simulate reads for this organism
this_genome = "refs_to_simulate_reads_from/moraxellacatarhalis_NZ_CP018059.fasta"

In [3]:
def mutate_sequence(sequence, pid):
    '''
    Given a sequence and a % ID threshold, mutate the sequence at random positions to produce
    a new sequence with the specified % ID. Return the new sequence string.
    '''
    
    swap_values = ['A','C','G','T']
    
    len_of_seq = len(sequence)
    num_bp_to_change = (np.round(len_of_seq) * (1-pid))
    all_bp_locs = [b for b in range(len_of_seq)]    
    bp_locs_to_change = random.sample(all_bp_locs, int(num_bp_to_change))
    
    # loop through indices of bases to change and swap out values
    new_seq = list(sequence)
    for bp in bp_locs_to_change:
        new_seq[bp] = random.sample([i for i in swap_values if i != new_seq[bp] ], 1)[0]
        
    return("".join(new_seq))

# generate a genome that is slightly different from anything in the DB already (99.8% similar to the reference)
records = list(SeqIO.parse(this_genome, "fasta"))         # read in the genome file
seq_to_simulate = mutate_sequence(records[0].seq, .998)   # mutate the sequence

# write the new genome output
with open("simulation_reference.fasta", 'w') as f: 
    f.write('>' + records[0].id + ' -- mutated\n' + str(seq_to_simulate) + '\n')
f.close()

In [4]:
# simulate sequences from the input file using InSilicoSeq
max_size = 100000
! iss generate --genomes simulation_reference.fasta --model miseq --output miseq_reads --n_reads {max_size}

INFO:iss.app:Starting iss generate
INFO:iss.app:Using kde ErrorModel
INFO:iss.util:Stitching input files together
INFO:iss.app:Using lognormal abundance distribution
INFO:iss.app:Using 2 cpus for read generation
INFO:iss.app:Generating 100000 reads
INFO:iss.app:Generating reads for record: NZ_CP018059.1
INFO:iss.util:Stitching input files together
INFO:iss.util:Stitching input files together
INFO:iss.util:Cleaning up
INFO:iss.app:Read generation complete


In [26]:
# pseudo-simulate 1 million reads by just 10x-ing the 100,000 simulated reads
! cat miseq_reads_R1.fastq miseq_reads_R1.fastq miseq_reads_R1.fastq miseq_reads_R1.fastq miseq_reads_R1.fastq miseq_reads_R1.fastq miseq_reads_R1.fastq miseq_reads_R1.fastq miseq_reads_R1.fastq miseq_reads_R1.fastq > miseq_reads_10x_R1.fastq
! mv miseq_reads_10x_R1.fastq miseq_reads_R1.fastq
! cat miseq_reads_R2.fastq miseq_reads_R2.fastq miseq_reads_R2.fastq miseq_reads_R2.fastq miseq_reads_R2.fastq miseq_reads_R2.fastq miseq_reads_R2.fastq miseq_reads_R2.fastq miseq_reads_R2.fastq miseq_reads_R2.fastq > miseq_reads_10x_R2.fastq
! mv miseq_reads_10x_R2.fastq miseq_reads_R2.fastq

In [27]:
max_size = 1000000

In [ ]:
metric_collection = {}

simulated_input_sizes = [1000, 10000, 100000, 1000000]
total_iterations = 5

# attempt to speed things up by creating the index just once at the outset
this_dump_fasta = this_dump + '.fasta'
! minimap2 -x sr -d ref_orig.mmi {this_dump_fasta} 
new_dump_fasta = this_dump + '.new.fasta'
! minimap2 -x sr -d ref_clustered.mmi {new_dump_fasta} 
        

for input_size in simulated_input_sizes:
    print("INPUT SIZE: " + str(input_size))

    metric_collection[input_size] = {'original_time': [], 'clustered_time': [], 'timediffx': [], 
                                     'original_unmapped_reads': [], 'clustered_unmapped_reads': [], 'unmapped_diff': []}

    print("generating subset data...")
    if(input_size < max_size):
        ! seqkit sample -s100 -n {input_size} miseq_reads_R1.fastq -o sub1.fastq
        ! seqkit sample -s100 -n {input_size} miseq_reads_R2.fastq -o sub2.fastq
    else:
        ! cp miseq_reads_R1.fastq sub1.fastq
        ! cp miseq_reads_R2.fastq sub2.fastq
    
    print("completed generating subset data")
    for i in range(total_iterations):
        
        print("ITERATION: " + str(i))

        # map to original dump
        #this_dump_fasta = this_dump + '.fasta'
        #! minimap2 -ax sr {this_dump_fasta} miseq_reads_R1.fastq miseq_reads_R2.fastq > aln.sam
        # separate indexing from alignment time
        #print("CREATING INDEX")
        #! minimap2 -x sr -d ref.mmi {this_dump_fasta} 
        print("RUNNING ALIGNMENT")
        start = time.time()  # start after indexing, but before alignment
        ! minimap2 -ax sr ref_orig.mmi sub1.fastq sub2.fastq > aln.sam
        end = time.time()   # end directly after alignment to capture ONLY alignment time
        print("ALIGNMENT COMPLETE, collecting stats")
        ! samtools stats aln.sam > alnstats.txt
        unmapped_reads_string = ! grep "reads unmapped:" alnstats.txt
        orig_unmapped_reads_count = int(unmapped_reads_string[0].split(':')[-1].strip())    
        orig_elapsed = end - start
        print('\n\n')
        print("unmapped_reads_count: " + str(orig_unmapped_reads_count))
        print("time elapsed: " + str(orig_elapsed))
        print('\n\n')


        # map to clustered dump
        #new_dump_fasta = this_dump + '.new.fasta'
        #! minimap2 -ax sr {new_dump_fasta} miseq_reads_R1.fastq miseq_reads_R2.fastq > aln.sam
        # separate indexing from alignment time
        #print("CREATING INDEX")
        #! minimap2 -x sr -d ref.mmi {new_dump_fasta} 
        print("RUNNING ALIGNMENT")
        start = time.time()  # start after indexing, but before alignment
        ! minimap2 -ax sr ref_clustered.mmi sub1.fastq sub2.fastq > aln.sam
        end = time.time()   # end directly after alignment to capture ONLY alignment time
        print("ALIGNMENT COMPLETE, collecting stats")
        ! samtools stats aln.sam > alnstats.txt
        unmapped_reads_string = ! grep "reads unmapped:" alnstats.txt
        unmapped_reads_count = int(unmapped_reads_string[0].split(':')[-1].strip())
        elapsed = end - start

        print('\n\n')
        print("unmapped_reads_count: " + str(unmapped_reads_count))
        print("time elapsed: " + str(elapsed))
        print('\n\n')
    
        
        metric_collection[input_size]['original_time'].append(orig_elapsed)
        metric_collection[input_size]['clustered_time'].append(elapsed)
        metric_collection[input_size]['original_unmapped_reads'].append(orig_unmapped_reads_count)
        metric_collection[input_size]['clustered_unmapped_reads'].append(unmapped_reads_count)
        metric_collection[input_size]['timediffx'].append(orig_elapsed/elapsed)
        metric_collection[input_size]['unmapped_diff'].append((orig_unmapped_reads_count - unmapped_reads_count)/input_size)
        
        

[M::mm_idx_gen::10.019*1.53] collected minimizers
[M::mm_idx_gen::12.501*1.81] sorted minimizers
[M::main::14.334*1.69] loaded/built the index for 19717 target sequence(s)
[M::mm_idx_stat] kmer size: 21; skip: 11; is_hpc: 0; #seq: 19717
[M::mm_idx_stat::14.359*1.69] distinct minimizers: 1884822 (17.20% are singletons); average occurrences: 57.418; average spacing: 5.996; total length: 648952447
[M::main] Version: 2.22-r1101
[M::main] CMD: minimap2 -x sr -d ref_orig.mmi ../experiment-4/moraxellacatarhalis_29497_dump.fasta
[M::main] Real time: 14.761 sec; CPU: 24.650 sec; Peak RSS: 3.503 GB
[M::mm_idx_gen::0.773*1.00] collected minimizers
[M::mm_idx_gen::0.882*1.24] sorted minimizers
[M::main::1.081*1.17] loaded/built the index for 7984 target sequence(s)
[M::mm_idx_stat] kmer size: 21; skip: 11; is_hpc: 0; #seq: 7984
[M::mm_idx_stat::1.102*1.17] distinct minimizers: 1513654 (36.42% are singletons); average occurrences: 3.633; average spacing: 6.032; total length: 33170697
[M::main] Vers

ALIGNMENT COMPLETE, collecting stats



unmapped_reads_count: 0
time elapsed: 0.4602961540222168



INPUT SIZE: 10000
generating subset data...
[INFO] sample by number
[INFO] loading all sequences into memory...
[INFO] 9979 sequences outputted
[INFO] sample by number
[INFO] loading all sequences into memory...
[INFO] 9979 sequences outputted
completed generating subset data
ITERATION: 0
RUNNING ALIGNMENT
[M::main::0.754*1.02] loaded/built the index for 19717 target sequence(s)
[M::mm_mapopt_update::0.754*1.02] mid_occ = 1000
[M::mm_idx_stat] kmer size: 21; skip: 11; is_hpc: 0; #seq: 19717
[M::mm_idx_stat::0.778*1.02] distinct minimizers: 1884822 (17.20% are singletons); average occurrences: 57.418; average spacing: 5.996; total length: 648952447
[M::worker_pipeline::13.381*2.87] mapped 19958 sequences
[M::main] Version: 2.22-r1101
[M::main] CMD: minimap2 -ax sr ref_orig.mmi sub1.fastq sub2.fastq
[M::main] Real time: 13.578 sec; CPU: 38.626 sec; Peak RSS: 1.259 GB
ALIGNMENT COMPLETE, co

ALIGNMENT COMPLETE, collecting stats



unmapped_reads_count: 4
time elapsed: 129.90127515792847



RUNNING ALIGNMENT
[M::main::0.148*1.15] loaded/built the index for 7984 target sequence(s)
[M::mm_mapopt_update::0.148*1.15] mid_occ = 1000
[M::mm_idx_stat] kmer size: 21; skip: 11; is_hpc: 0; #seq: 7984
[M::mm_idx_stat::0.166*1.13] distinct minimizers: 1513654 (36.42% are singletons); average occurrences: 3.633; average spacing: 6.032; total length: 33170697
[M::worker_pipeline::12.330*2.99] mapped 166114 sequences
[M::worker_pipeline::14.291*2.97] mapped 33886 sequences
[M::main] Version: 2.22-r1101
[M::main] CMD: minimap2 -ax sr ref_clustered.mmi sub1.fastq sub2.fastq
[M::main] Real time: 14.314 sec; CPU: 42.509 sec; Peak RSS: 0.417 GB
ALIGNMENT COMPLETE, collecting stats



unmapped_reads_count: 7
time elapsed: 14.46250295639038



ITERATION: 1
RUNNING ALIGNMENT
[M::main::0.749*1.03] loaded/built the index for 19717 target sequence(s)
[M::mm_mapopt_update::0.750*1.03] mid_occ = 1000


[M::mm_idx_stat::0.184*1.36] distinct minimizers: 1513654 (36.42% are singletons); average occurrences: 3.633; average spacing: 6.032; total length: 33170697
[M::worker_pipeline::12.951*3.01] mapped 166114 sequences
[M::worker_pipeline::25.548*3.03] mapped 166114 sequences
[M::worker_pipeline::38.035*3.03] mapped 166114 sequences
[M::worker_pipeline::50.856*3.03] mapped 166114 sequences
[M::worker_pipeline::64.304*3.02] mapped 166114 sequences
[M::worker_pipeline::77.529*3.00] mapped 166114 sequences
[M::worker_pipeline::77.546*3.00] mapped 3316 sequences
[M::main] Version: 2.22-r1101
[M::main] CMD: minimap2 -ax sr ref_clustered.mmi sub1.fastq sub2.fastq
[M::main] Real time: 77.574 sec; CPU: 232.743 sec; Peak RSS: 0.711 GB
ALIGNMENT COMPLETE, collecting stats



unmapped_reads_count: 20
time elapsed: 77.80833292007446



ITERATION: 1
RUNNING ALIGNMENT
[M::main::0.768*1.09] loaded/built the index for 19717 target sequence(s)
[M::mm_mapopt_update::0.768*1.09] mid_occ = 1000
[M::mm_idx_st

In [ ]:
print(metric_collection)

In [ ]:
# FOR FUTURE USE -- MAKE A BOXPLOR OF THE metric_collection data

time_dict = dict(zip(simulated_input_sizes, [metric_collection[i]['timediffx'] for i in simulated_input_sizes]))

labels, data = time_dict.keys(), time_dict.values()

plt.boxplot(data)
plt.xticks(range(1, len(labels) + 1), labels)
plt.ylabel('Time Difference')
plt.xlabel('Number of reads to align')
plt.title(this_dump.split('/')[-1])
plt.show()
